In [5]:
import tensorflow as tf
tf.python.control_flow_ops = tf

import os

os.environ['CUDA_VISIBLE_DEVICES']=str(1)

import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
import csv
from keras.models import Sequential, Model
from keras.layers import Conv2D, ConvLSTM2D, Dense, MaxPooling2D, Dropout, Flatten, Reshape, merge, Input
from keras.optimizers import Adam
from keras_tqdm import TQDMNotebookCallback
from generator import generator
from car_models import *
from sklearn.utils import shuffle
from tqdm import tqdm_notebook


from get_images import get_images

datadirs=['/notebooks/udacity/new_training/map1_backward/',
                 '/notebooks/udacity/new_training/map1_forward/',
                 '/notebooks/udacity/new_training/map1_recovery_backward/',
                 '/notebooks/udacity/new_training/map1_recovery_forward/',
                 '/notebooks/udacity/new_training/map2_forward/',
                 '/notebooks/udacity/new_training/map2_backward/',
                 '/notebooks/udacity/new_training/map2_recovery_forward/',
                 '/notebooks/udacity/new_training/map2_recovery_backward/',
                   '/notebooks/udacity/new_training/map1_error_correction/',
                   '/notebooks/udacity/new_training/map2_error_correction/'
         ]

images=get_images(datadirs,0.08)
image_names_full, y_data_full = images.img.values, images.real.values

size=(40,80)

def preprocessing_resize(im):
    return cv2.resize(cv2.cvtColor(im, cv2.COLOR_BGR2YUV),(size[1],size[0]))

def preprocessing_yuv(im):
    return cv2.cvtColor(im, cv2.COLOR_BGR2YUV)

def proc_img(img): # input is 160x320x3
    img = img[59:138:2, 0:-1:2, :] # select vertical region and take each second pixel to reduce image dimensions
    img = (img / 127.5) - 1.0 # normalize colors from 0-255 to -1.0 to 1.0
    return img # return 40x160x3 image

names_train, names_valid, y_train, y_valid = train_test_split(image_names_full, y_data_full, test_size=0.02,\
                            random_state=0)

names_valid, names_test, y_valid, y_test = train_test_split(names_valid, y_valid, test_size=0.5,\
                                        random_state=0)

inverse_train=[0 for i in y_train]+[1 for i in y_train]


names_train=  np.concatenate([names_train,names_train])
y_train    =   np.concatenate([y_train,y_train])

names_train,y_train,inverse_train=shuffle(names_train,y_train,inverse_train) 





In [21]:
import os
import pickle


def train_model(get_model, preprocessing, name, optimizer = 'adam',nb_epoch=100):
    #print ('Training model %s' % name)
    
    model=get_model()
    model.compile(loss='mse', optimizer=optimizer, metrics=['mean_squared_error'])
    
    if not os.path.exists('results/logs_%s'%name):
        os.mkdir('results/logs_%s'%name)
    
    
    checkpoint = ModelCheckpoint("results/model_%s.h5" % name, monitor='val_mean_squared_error', verbose=1,
                              save_best_only=False, mode='min')
    early_stop = EarlyStopping(monitor='val_mean_squared_error',\
                               min_delta=0.001, patience=3,
                                verbose=False, mode='min')

    history = History()

    tb=TensorBoard(log_dir='results/logs_%s'%name, histogram_freq=0, write_graph=True, write_images=False)
    
    train_gen=generator2(names_train, y_train, batch_size=64, preprocessing=preprocessing, inverse=inverse_train)
    valid_gen=generator(names_valid, y_valid, batch_size=64, preprocessing=preprocessing)
    test_gen= generator (names_test, y_test, batch_size=64,preprocessing=preprocessing)

    q=model.fit_generator(train_gen, \
                    samples_per_epoch=len(names_train),\
                    nb_epoch=nb_epoch,\
                    verbose=False,\
                    validation_data=valid_gen, \
                    nb_val_samples=len(names_valid),\
                    callbacks=[checkpoint, early_stop,history,tb,TQDMNotebookCallback()]\
                   )
    
    f=open('results/history_%s.pk1' % name,'wb')
    pickle.dump(history.history,f,-1)
    f.close()
    
    model.load_weights("results/model_%s.h5" % name)
    
    res=model.evaluate_generator(test_gen,val_samples=len(names_test))[0]
    return q,res

In [ ]:
train_model(modified_vgg, proc_img, 'modified_vgg', optimizer=Adam(lr=0.0001), nb_epoch=10)

Training model modified_vgg
Instructions for updating:
Please switch to tf.summary.merge_all.


The installed widget Javascript is the wrong version.
The installed widget Javascript is the wrong version.


Epoch 00003: saving model to results/model_modified_vgg.h5


The installed widget Javascript is the wrong version.


Epoch 00004: saving model to results/model_modified_vgg.h5


...

Epoch 00006: saving model to results/model_modified_vgg.h5


The installed widget Javascript is the wrong version.


Epoch 00007: saving model to results/model_modified_vgg.h5


The installed widget Javascript is the wrong version.


KeyboardInterrupt: 

In [23]:
grid=[{'get_model':modified_lenet,'preprocessing':lambda x:x,       'optimizer':'adam','name':'modified_lenet'},\
      {'get_model':modified_vgg,  'preprocessing': proc_img,        'optimizer':'adam','name':'modified_vgg'},\
      {'get_model':nvidia_net,    'preprocessing':preprocessing_yuv,'optimizer':'adam','name':'nvidia_net'},\
      {'get_model':inception,     'preprocessing':lambda x:x,       'optimizer':'adam','name':'inception'},\
      {'get_model':vgg16,         'preprocessing':lambda x:x,       'optimizer':'adam','name':'vgg16'} ]

In [ ]:
results=[train_model(**a) for a in tqdm_notebook(grid, desc='models')]

The installed widget Javascript is the wrong version.


Instructions for updating:
Please switch to tf.summary.merge_all.


The installed widget Javascript is the wrong version.
The installed widget Javascript is the wrong version.
